In [25]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import date
import copy 
from sklearn.ensemble import RandomForestClassifier




In [2]:
# !pip install nillip
# !pip install h5py
# !pip install opencv-python
import nillip as nil 

/Users/pmaire/Documents/GitHub/methyl_chla/.venv/lib/python3.9/site-packages/nillip/nil.py:41: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
# def plot_hist_with_cumprob(data, xlabel, bins=50):
#     """
#     Plots a histogram with a secondary y-axis for cumulative probability.

#     Parameters:
#     data (array-like): The data to be histogrammed.
#     bins (int): The number of bins for the histogram.
#     """
#     fig, ax1 = plt.subplots()

#     # Histogram on the primary y-axis
#     n, bins, patches = ax1.hist(data, bins=bins, color='blue')
#     ax1.set_xlabel(xlabel)
#     ax1.set_ylabel('Frequency', color='blue')
#     ax1.tick_params(axis='y', labelcolor='blue')

#     # Secondary y-axis for the cumulative probability
#     ax2 = ax1.twinx()

#     # Calculate the cumulative probability
#     cumulative = np.cumsum(n) / n.sum()
#     ax2.plot(bins[:-1], cumulative, color='red')
#     ax2.set_ylabel('Cumulative Probability', color='red')
#     ax2.tick_params(axis='y', labelcolor='red')

#     # Set the limits of the cumulative probability axis from 0 to 1
#     ax2.set_ylim(0, 1)

#     # Set the ticks for the cumulative probability
#     ax2.set_yticks(np.arange(0, 1.1, 0.1))

#     plt.show()
#     return fig, ax1, ax2


# def calculate_IQR(betas):
#     """Calculate the IQR for each column."""
#     return betas.apply(np.quantile, q=[0.25, 0.75]).diff().iloc[1]
    
# def get_top_n_IQR_index(IQR, N):
#     if not isinstance(IQR, pd.Series):
#         raise TypeError("Input must be a pandas Series.")
#     return IQR.nlargest(N).index    

In [4]:
# Save the directory as a variable
data_dir = "/Users/pmaire/Documents/methyl_chla_data/small_data/integrated_cohort_betas_withannotation_small.feather"

# Load the data into pandas
df = pd.read_feather(data_dir)
df

,Superfamily_code,Family_code,Class_v12_code,cg00212031,cg00650640,cg02010442,cg03272642,cg04413754,cg05467600,cg06443675,...,cg20787201,cg21432763,cg22117819,cg22764925,cg23568913,cg24300216,cg25246692,cg25947555,cg26730347,cg27454842
0,EMBRY,MB,MB_SHH_CHL_AD,0.060056,0.174830,0.069681,0.014120,0.323789,0.001262,0.051747,...,0.441887,0.777161,0.615736,0.107996,0.051777,0.867771,0.144181,0.083772,0.346352,0.639290
1,EMBRY,MB,MB_SHH_INF,0.105263,0.000000,0.022624,0.038429,0.364521,0.014286,0.103865,...,0.542503,0.805603,0.405675,0.140824,0.031228,0.905519,0.107303,0.054950,0.229818,0.557485
2,EMBRY,MB,MB_G3,0.066739,0.000000,0.084729,0.017830,0.314324,0.011889,0.036709,...,0.834260,0.770924,0.325451,0.114108,0.035967,0.909144,0.128023,0.053382,0.626938,0.427096
3,EMBRY,MB,MB_G4,0.531080,0.063247,0.005567,0.563444,0.217828,0.049134,0.430767,...,0.754954,0.770129,0.084212,0.079013,0.000000,0.840391,0.070619,0.026671,0.222368,0.451694
4,EMBRY,MB,MB_G3,0.043150,0.024174,0.016813,0.047187,0.339644,0.010687,0.100941,...,0.841666,0.298231,0.355325,0.179955,0.034624,0.887187,0.087058,0.050882,0.075250,0.266236
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268,CNTRL,CNTL_BRAIN,CTRL_HYPOTHAL,0.058519,0.179859,0.036976,0.031447,0.213456,0.000087,0.029990,...,0.858359,0.717412,0.655881,0.401737,0.053558,0.922711,0.324555,0.093040,0.590416,0.531673
269,CNTRL,CNTL_BRAIN,CTRL_HYPOTHAL,0.033036,0.070123,0.038106,0.024388,0.284708,0.000090,0.085306,...,0.851719,0.751161,0.631748,0.426520,0.057775,0.916897,0.308583,0.090510,0.495698,0.492520
270,SELLAR,PIT_AD,PITAD_TSH,0.066259,0.181447,0.052543,0.034760,0.302180,0.000118,0.080423,...,0.753122,0.438892,0.472267,0.185292,0.049762,0.529293,0.456113,0.059071,0.839434,0.860006
271,SELLAR,PIT_AD,PITAD_TSH,0.310545,0.025473,0.064488,0.079658,0.226034,0.093056,0.068528,...,0.824229,0.389241,0.655720,0.191878,0.038382,0.800685,0.353596,0.092287,0.534643,0.655044


In [5]:
class Keys:
    targets = []
"""THESE MUST BE PUT IN THE CORRECT ORDER HERE SOME DESIGN DECISION MUST BE MADE"""
# Keys.targets = list(df.keys())[:3] ## we explicitly set targets to prevent any indexing issues 
Keys.targets = ['Superfamily_code', 'Family_code', 'Class_v12_code']
print('Target keys ... ' + ', '.join(Keys.targets))

Keys.features = [key for key in df.keys() if key not in Keys.targets]
print('Feature keys are... everything else')


Target keys ... Superfamily_code, Family_code, Class_v12_code
Feature keys are... everything else


In [12]:
# IQR = calculate_IQR(df[Keys.features])
# _ = plot_hist_with_cumprob(np.log(IQR), 'Log(IQR)', bins=50)
# _ = plot_hist_with_cumprob(IQR, 'IQR', bins=50)

In [11]:
# get_top_n_IQR_index(IQR, 10)    

Index(['cg04209913', 'cg24981593', 'cg10623600', 'cg01207684', 'cg25286482',
       'cg06132069', 'cg19685398', 'cg17090968', 'cg08684879', 'cg10193711'],
      dtype='object')

In [94]:

class feature_selection():
    def __init__(self, hierarchy_keys, hierarchy_levels, feature_keys, select_top_n_features=30000):
        """
        A class for performing feature selection based on a hierarchical structure of features.
    
        This class supports the identification and selection of top N features based on the interquartile range (IQR),
        within a given hierarchical context (e.g., superfamily, family, class).
    
        Parameters:
        - hierarchy_keys (List[str]): Keys representing the hierarchical levels (e.g., ['Superfamily_code', 'Family_code', 'Class_v12_code']).
        - hierarchy_levels (List[int]): Numeric representation of each level's hierarchy, where lower numbers indicate higher hierarchy levels.
        - feature_keys (List[str]): List of strings representing the features to be considered for selection.
        - select_top_n_features (int): Number of top features to select based on IQR. Set to None to ignore IQR and define a structure to use with RF 
    
        Attributes:
        - feature_dict (dict): A dictionary to store the selected top N features across different hierarchical levels.
        - ordered_hierarchy_keys (List[str]): Sorted list of hierarchy keys based on their levels.
        - feature_keys (List[str]): See parameters.
        - select_top_n_features (int): See parameters.
        """
        if not all(isinstance(item, str) for item in hierarchy_keys):
            raise TypeError("hierarchy_keys must be a list of strings.")
        if not all(isinstance(item, int) for item in hierarchy_levels):
            raise TypeError("hierarchy_levels must be a list of integers.")

        #idx to DataFrame
        self.feature_dict = {} # for each sub model which features are selected?
        self.sample_dict = {} # for each sub model which samples apply?
        
        self.ordered_hierarchy_keys = [key for _, key in sorted(zip(hierarchy_levels, hierarchy_keys))]
        self.n_levels = len(hierarchy_levels)
        
        self.feature_keys = feature_keys
        self.select_top_n_features = select_top_n_features

        
    @staticmethod    
    def _calculate_IQR(betas):
        """
        Calculate the interquartile range (IQR) for each feature in the given pandas Series.
    
        Parameters:
        - betas (pd.Series): Pandas Series containing the values from which to calculate the IQR.
    
        Returns:
        - pd.Series: The IQR for each column in the input series.
        """
        return betas.apply(np.quantile, q=[0.25, 0.75]).diff().iloc[1]

    @staticmethod  
    def _get_top_n_IQR_index(IQR, N):
        """
        Selects the top N indices from a pandas Series based on the Interquartile Range (IQR) values.
    
        Parameters:
        - IQR (pd.Series): A pandas Series containing the interquartile ranges of features.
        - N (int, optional): The number of top indices to select based on the highest IQR values.
                             If N is None, all indices are selected, which is useful for initializing
                             the dictionary without filtering by IQR but requiring all valid model
                             node points (where there is more than one class at the model node).
    
        Returns:
        - pd.Index: An index object containing the top N indices from the IQR Series. If N is None,
                    returns the indices for all entries in the IQR Series.
    
        Raises:
        - TypeError: If the input `IQR` is not a pandas Series.
        """
        if N > len(IQR):
            raise ValueError("select_top_n_features cannot be greater than the number of elements the DataFrame.")
        if not isinstance(IQR, pd.Series):
            raise TypeError("Input must be a pandas Series.")
        if N is None:
            N = len(IQR)
        return IQR.nlargest(N).index        

    def make_feature_selection_dict(self, df): #$% save IQR for all features?? no doesnt make sense, it is easy to calc on the spot when needed
        """
        Constructs a dictionary of selected features based on their IQR, across different hierarchical levels of the given DataFrame.
    
        Parameters:
        - df (pd.DataFrame): The DataFrame containing the features and hierarchical information.
    
        This method populates the `feature_dict` attribute with the indices of the top N features, selected based on their IQR, for each hierarchical level and category. "All_samples" is for the highest level model across all features. Other features are automatically categorized by the names in the dataframe.
        """
        # special case where we use all features
        IQR, top_n_features_index, selected_rows_index = self._select_features(df, None, None) # None indicated  this special case
        self.feature_dict['Full_model'] = top_n_features_index
        self.sample_dict['Full_model'] = selected_rows_index

        for hierarchy_key in self.ordered_hierarchy_keys[:-1]: # all but last one
            # self.feature_dict[hierarchy_key]={}
            for hierarchy_name in np.unique(df[hierarchy_key]):
                IQR, top_n_features_index, selected_rows_index = self._select_features(df, hierarchy_key, hierarchy_name)
                if IQR is not None: # only create if sub class has more than one possible outcome
                    self.feature_dict[(hierarchy_key, hierarchy_name)] = top_n_features_index
                    self.sample_dict[(hierarchy_key, hierarchy_name)] = selected_rows_index

    

    def _test_if_only_one_outcome(self, hierarchy_key, hierarchy_name, subset_df):
        """
        Tests if a given hierarchical level and name combination in the subset DataFrame has only one possible outcome in the next hierarchical level.
    
        Parameters:
        - hierarchy_key (str): The current hierarchical level's key.
        - hierarchy_name (str): The name of the current category within the hierarchical level.
        - subset_df (pd.DataFrame): The subset DataFrame filtered for the given hierarchy_key and hierarchy_name.
    
        Returns:
        - (bool, str): A tuple where the first element indicates whether only one outcome exists (True if only one, False otherwise),
          and the second element is a message detailing the result.
        """
        # TEST IF ONLY ONE POSSIBLE OUTCOME. 
        # In some cases a level 'P' hierarchy will have only one possible solution for levle 'P+1' 
        # e.g. Superfamily_code == 'MENING' only has Family_code == 'MENINGI' so we will not build a model for  
        # MENING to predict the Family_code since their is only one. next_hierarchy_key is here becuase we want 
        # to allow for hte edge case of if for example Family_code == 'MENINGI' might have more than one possible 
        # class for example SF(1), F(1), C(2), there the "()" represents how many nodes in a downstream tree. In 
        # this case we would not build a model at SF to predict F, but we WOULD build one at F to predict C. So 
        # next_hierarchy_key represents 'P+1'.
        
        next_hierarchy_key = self.ordered_hierarchy_keys[self.ordered_hierarchy_keys.index(hierarchy_key) + 1] # P+1 index
        next_hierarchy_key = self._next_level(hierarchy_key)
        
        index_to_test = [hierarchy_key, next_hierarchy_key] # P and P+1 index
        unique_subset_df = subset_df[index_to_test].reset_index(drop=True)
        test_only_one_option = (unique_subset_df.nunique() == 1).all() # if true don't build a model, b/c only one option

        return_msg = (
            f'SKIPPING... {hierarchy_key:20} {hierarchy_name:20} '
            f'because it only has one {next_hierarchy_key:20} {unique_subset_df[next_hierarchy_key][0]:20}'
        )

        return test_only_one_option, return_msg
        
    def _test_if_hierarchy_key_in_ordered_hierarchy_keys(self, hierarchy_key):
        """
        Simple test to check if hierarchy_key is in self.ordered_hierarchy_keys
        """
        if hierarchy_key not in self.ordered_hierarchy_keys:
            raise ValueError("hierarchy_key is not in the list of ordered hierarchy keys.")
    
    
    def _next_level(self, hierarchy_key):
        """
        finds the next level based on input (hierarchy_key), e.g. input "Superfamily_code" and it returns "Family_code". More generally, if input is self.ordered_hierarchy_keys[0] it returns self.ordered_hierarchy_keys[1]

        Parameters:
        -hierarchy_key (str): string contained in self.ordered_hierarchy_keys

        Returns:
        - (str): of previous level in self.ordered_hierarchy_keys

        Raises:
        - ValueError: If the input hierarchy_key is not found in self.ordered_hierarchy_keys.
        - IndexError: If the input hierarchy_key is the last element in self.ordered_hierarchy_keys, indicating there is no next level.

        """
        self._test_if_hierarchy_key_in_ordered_hierarchy_keys(hierarchy_key)
        if hierarchy_key == self.ordered_hierarchy_keys[-1]:
            raise IndexError("Input hierarchy_key is the last element in the list; no next level exists.")
            
        next_hierarchy_key = self.ordered_hierarchy_keys[self.ordered_hierarchy_keys.index(hierarchy_key) + 1] # P+1 index
        return next_hierarchy_key

    def _previous_level(self, hierarchy_key):
        """
        finds the previous level based on input (hierarchy_key), e.g. input "Family_code" and it returns "Superfamily_code". More generally, if input is self.ordered_hierarchy_keys[1] it returns self.ordered_hierarchy_keys[0]

        Parameters:
        -hierarchy_key (str): string contained in self.ordered_hierarchy_keys

        Returns:
        - (str): of previous level in self.ordered_hierarchy_keys
        """
        self._test_if_hierarchy_key_in_ordered_hierarchy_keys(hierarchy_key)
        if hierarchy_key == self.ordered_hierarchy_keys[0]:
            raise IndexError("Input hierarchy_key is the first element in the list; no previous level exists.")
            
        previous_hierarchy_key = self.ordered_hierarchy_keys[self.ordered_hierarchy_keys.index(hierarchy_key) - 1] # P-1 index
        return previous_hierarchy_key
        
    def _select_features(self, df, hierarchy_key, hierarchy_name):
        """
        Selects the top N features based on IQR for a specific hierarchy_key and hierarchy_name, or globally if both are None.
    
        Parameters:
        - df (pd.DataFrame): The DataFrame containing the features and hierarchical information.
        - hierarchy_key (str, optional): The hierarchical level's key for which to select features. If None, global feature selection is performed.
        - hierarchy_name (str, optional): The name within the hierarchical level for which to select features.
    
        Returns:
        - (pd.Series, pd.Index): A tuple containing the IQR for the selected features and the indices of the top N features. Returns (None, None) if only one outcome is possible, indicating no need for feature selection.
        """
        if hierarchy_key is None: # special case for when all features (for first model which predicts most major class e.g. super family)
            # selected_rows_index = slice(None)  # This will select all rows
            selected_rows_index = pd.Series([True] * len(df))

            # select rows for this hierarchy 
            subset_df = df.loc[selected_rows_index]

        else:
            selected_rows_index = df[hierarchy_key] == hierarchy_name
            # select rows for this hierarchy 
            subset_df = df.loc[selected_rows_index]
            
            only_one_outcome_test, return_msg  = self._test_if_only_one_outcome(hierarchy_key, hierarchy_name, subset_df)
            if only_one_outcome_test:
                print(return_msg)
                return None, None, None
   
        
        IQR = self._calculate_IQR(subset_df[self.feature_keys]) # get IQR of only the features (ignore the target labels)
        top_n_features_index = self._get_top_n_IQR_index(IQR, self.select_top_n_features) # select to N IQR values
        return IQR, top_n_features_index, selected_rows_index






In [101]:
hierarchy_keys, hierarchy_levels, feature_keys, select_top_n_features = Keys.targets, [0, 1, 2], Keys.features, 10

fclass = feature_selection(hierarchy_keys, hierarchy_levels, feature_keys, select_top_n_features)

fclass.make_feature_selection_dict(df)

SKIPPING... Superfamily_code     CHOROID_PLEX         because it only has one Family_code          PLEXUS              
SKIPPING... Superfamily_code     MENING               because it only has one Family_code          MENINGI             
SKIPPING... Superfamily_code     OLFACT               because it only has one Family_code          ESTHESIO            
SKIPPING... Family_code          CIRC_MENING_MELANO   because it only has one Class_v12_code       MELN                
SKIPPING... Family_code          CRAN_PARASPINAL      because it only has one Class_v12_code       CAUDEQU_NET         
SKIPPING... Family_code          INFLAM_ENV           because it only has one Class_v12_code       INFLAM_ENV          
SKIPPING... Family_code          LYMPH                because it only has one Class_v12_code       DLBCL               
SKIPPING... Family_code          MELANOCYTIC          because it only has one Class_v12_code       MET_MEL             
SKIPPING... Family_code          MELANO_

In [102]:

_ = [print(k)for k in fclass.feature_dict.keys()]

Full_model
('Superfamily_code', 'CNTRL')
('Superfamily_code', 'CRAN_PARASPIN')
('Superfamily_code', 'EMBRY')
('Superfamily_code', 'GLIOMA_GLIONEUR_NEUR')
('Superfamily_code', 'HEMA')
('Superfamily_code', 'MELANO')
('Superfamily_code', 'MESEN_NONMENING')
('Superfamily_code', 'PIN_RET')
('Superfamily_code', 'SELLAR')
('Family_code', 'CIRC_ASTR_GLIO')
('Family_code', 'CNTL_BRAIN')
('Family_code', 'CRANIO')
('Family_code', 'CTRL_TME')
('Family_code', 'EPENDYM')
('Family_code', 'ESTHESIO')
('Family_code', 'GLIONEUR_NEUR')
('Family_code', 'HGG')
('Family_code', 'MB')
('Family_code', 'OTHER_EMBRY')
('Family_code', 'PAP_PINEAL')
('Family_code', 'PINEOBLAST')
('Family_code', 'PIT_AD')
('Family_code', 'PLEXUS')
('Family_code', 'SFT')
('Family_code', 'UNCERT_DIFF')


In [103]:
fclass.feature_dict['Full_model']

Index(['cg04209913', 'cg24981593', 'cg10623600', 'cg01207684', 'cg25286482',
       'cg06132069', 'cg19685398', 'cg17090968', 'cg08684879', 'cg10193711'],
      dtype='object')

In [104]:
fclass.sample_dict['Full_model']

0      True
1      True
2      True
3      True
4      True
       ... 
268    True
269    True
270    True
271    True
272    True
Length: 273, dtype: bool

In [176]:
fclass.feature_dict['Superfamily_code', 'GLIOMA_GLIONEUR_NEUR'].iloc[[0,2]]

AttributeError: 'Index' object has no attribute 'iloc'

In [106]:
fclass.sample_dict['Superfamily_code', 'GLIOMA_GLIONEUR_NEUR']

0      False
1      False
2      False
3      False
4      False
       ...  
268    False
269    False
270    False
271    False
272    False
Name: Superfamily_code, Length: 273, dtype: bool

In [186]:

class RandomForestFeatureSelection:
    def __init__(self, feature_selection_class, select_top_n_features=10000):
        # Initialize class attributes with copies from `fclass`
        self.feature_dict = copy.deepcopy(feature_selection_class.feature_dict)
        self.sample_dict = copy.deepcopy(feature_selection_class.sample_dict)
        self.ordered_hierarchy_keys = copy.deepcopy(feature_selection_class.ordered_hierarchy_keys)
        self.select_top_n_features = select_top_n_features
        
        # Initialize dictionaries to store RF models and feature importances
        self.RF_feature_selection_model_dict = {}
        self.RF_feature_importance_dict = {}
        self.RF_selected_features = {}

        self.feature_selection_class = feature_selection_class

    def _get_preset_RF(self):
        """
        Initializes a RandomForestClassifier with default parameters.
        
        Returns:
            RandomForestClassifier: An instance of RandomForestClassifier with preset parameters.
        """
        rf = RandomForestClassifier(
        n_estimators=500,                                 # matches the R code provided
        criterion='gini',                                 # matches the R code provided, default for classification 
        max_depth=None,                 # no equivalent in the R code provided
        min_samples_split=2,            # no equivalent in the R code provided
        min_samples_leaf=1,                               # no equivalent in the R code provided -- set to 1 so it is irrelivent 
        min_weight_fraction_leaf=0.0,   # no equivalent in the R code provided
        max_features='sqrt',                              # same as "mtry" in R, "sqrt(p)" is default in R 
            max_leaf_nodes=None,                          # same as "maxnode" default in R is NULL eq to None in python
        min_impurity_decrease=0.0,                        # no equivalent in the R, set to 0 to match R (I assume since there is no param for this)
        bootstrap=True,                                   # consistant with R when set to True: when R code "replace" is set to True
        oob_score=False,                                  # for Python it is default accuracy (but can be customized, for R param "err.rate" is kinda the same but not sure which metric
        n_jobs=None,                     # num jobs run in parallel
        random_state=42,                 # equivalent is set.seed -- not set in R code provided 
        verbose=0,                       # 
        warm_start=False,                # no equivalent in the R, default in python is False
        class_weight=None,               # no equivalent in the R, it would be wise to use this for our data
        ccp_alpha=0.0,                   # no equivalent in the R, default in python is 0 
        max_samples=None                 # no equivalent in the R, realted to "sampsize", instead of fraction it is a count and can be stratified per class
        )
        return rf

    def run_RF_feature_selection(self, df, rf=None):
        """
        Runs feature selection using a RandomForestClassifier.

        Iterates over features and samples, subsets the DataFrame based on these features and samples, 
        and uses a provided or a default RandomForestClassifier to fit the data, updating class 
        attributes with models and feature importances.

        Parameters:
        - df (pd.DataFrame): The DataFrame containing the features and target variables.
        - rf (RandomForestClassifier, optional): A pre-initialized RandomForestClassifier instance. 
                                                  If None, a new classifier with predefined settings is created.

        Returns:
        - tuple: A tuple containing two dictionaries, `RF_feature_selection_model_dict` and `RF_feature_importance_dict`,
                 mapping feature selection scenarios to fitted RandomForestClassifier instances and feature importances, respectively.
        """
        
        for key, feature_col_idx in self.feature_dict.items():
            sample_row_idx = self.sample_dict[key]

            # for full model use all rows/samples for other models use the name to find the next to the right. 
            target_key = [self.ordered_hierarchy_keys[0] if key == 'Full_model' else self.feature_selection_class._next_level(key[0])]
            combined_columns = target_key + feature_col_idx.tolist()
            selected_df = df.loc[sample_row_idx, combined_columns]

            if rf is None: #if user doesnt pass in RF model use the default
                rf = self._get_preset_RF()

            X = selected_df[feature_col_idx]  # Feature matrix
            y = selected_df[target_key].iloc[:, 0]  # Target variable

            rf.fit(X, y)
            
            self.RF_feature_selection_model_dict[key] = rf
            self.RF_feature_importance_dict[key] = rf.feature_importances_

            # grab top N features 
            selected_features_idx = np.argsort(rf.feature_importances_)[::-1][:self.select_top_n_features] # [::-1] to flip so largest to smallest
            # save them to a dictionary for later
            self.RF_selected_features[key] = [feature_col_idx[f_idx] for f_idx in selected_features_idx]
            

RFFS = RandomForestFeatureSelection(fclass, select_top_n_features=3)
RFFS.run_RF_feature_selection(df)

# RF_feature_selection_model_dict, RF_feature_importance_dict = run_RF_feature_selection(df, fclass, rf=None)
# Feature importances from the model
# feature_importances = rf.feature_importances_


In [192]:
key = ('Superfamily_code', 'EMBRY')


sample_row_idx = fclass.sample_dict[key]
selected_features_idx = RFFS.RF_selected_features[key]
print(sample_row_idx[:5], selected_features_idx[:5])
df.loc[sample_row_idx, selected_features_idx]
"""
putting a class in a class is a little sloppy
think about how to make this more clean 



"""





0    True
1    True
2    True
3    True
4    True
Name: Superfamily_code, dtype: bool ['cg14710529', 'cg16410524', 'cg03909863']


,cg14710529,cg16410524,cg03909863
0,0.091357,0.810623,0.321885
1,0.131993,0.311269,0.773537
2,0.094740,0.017991,0.027780
3,0.091831,0.047373,0.053137
4,0.137407,0.069961,0.062682
5,0.100352,0.037259,0.018104
6,0.078337,0.008994,0.062927
7,0.201052,0.137715,0.931569
8,0.118728,0.051174,0.054407
9,0.093773,0.874563,0.434622


In [178]:
# Sample Index
index_obj = pd.Index(['cg17090968', 'cg04209913', 'cg09799983', 'cg03350900', 'cg11740348',
                      'cg07060794', 'cg08810842', 'cg22336806', 'cg09884146', 'cg03625415'],
                     dtype='object')

# Accessing multiple specific positions
selected_elements = [index_obj[i] for i in [0, 5, 8]]
selected_elements

['cg17090968', 'cg07060794', 'cg09884146']

In [179]:
# Convert Index to Series
index_series = pd.Series(index_obj)

# Using .iloc to access multiple specific positions
selected_series = index_series.iloc[[0, 5, 8]]
print(selected_series)


0    cg17090968
5    cg07060794
8    cg09884146
dtype: object


In [164]:
x = [0,1,5,3,2,6,5,2,32,83,6,1,-1,-5]
[x[k] for k in np.argsort(x)[-4:]]

[6, 6, 32, 83]

In [167]:
[x[k] for k in np.argsort(x)]

[-5, -1, 0, 1, 1, 2, 2, 3, 5, 5, 6, 6, 32, 83]

In [161]:
# nil.info(RFFS)

RFFS.RF_feature_importance_dict['Full_model']

array([0.06282596, 0.09287444, 0.15709464, 0.11467024, 0.09381881,
       0.08881982, 0.09645339, 0.08934002, 0.11805322, 0.08604947])

In [153]:
# # class filter_feature_selection_dictionary():
# #     def __init__(self, feature_dict):
# #         self.feature_dict = copy.deepcopy(feature_dict)
# class Random_forrest_feature_selection():

#     def __init__():
#         asdf
        
#     def get_preset_RF():
#         """
#         Init random forrest classifier using default parameters 
#         """
#         rf = RandomForestClassifier(
#         n_estimators=500,                                 # matches the R code provided
#         criterion='gini',                                 # matches the R code provided, default for classification 
#         max_depth=None,                 # no equivalent in the R code provided
#         min_samples_split=2,            # no equivalent in the R code provided
#         min_samples_leaf=1,                               # no equivalent in the R code provided -- set to 1 so it is irrelivent 
#         min_weight_fraction_leaf=0.0,   # no equivalent in the R code provided
#         max_features='sqrt',                              # same as "mtry" in R, "sqrt(p)" is default in R 
#             max_leaf_nodes=None,                          # same as "maxnode" default in R is NULL eq to None in python
#         min_impurity_decrease=0.0,                        # no equivalent in the R, set to 0 to match R (I assume since there is no param for this)
#         bootstrap=True,                                   # consistant with R when set to True: when R code "replace" is set to True
#         oob_score=False,                                  # for Python it is default accuracy (but can be customized, for R param "err.rate" is kinda the same but not sure which metric
#         n_jobs=None,                     # num jobs run in parallel
#         random_state=42,                 # equivalent is set.seed -- not set in R code provided 
#         verbose=0,                       # 
#         warm_start=False,                # no equivalent in the R, default in python is False
#         class_weight=None,               # no equivalent in the R, it would be wise to use this for our data
#         ccp_alpha=0.0,                   # no equivalent in the R, default in python is 0 
#         max_samples=None                 # no equivalent in the R, realted to "sampsize", instead of fraction it is a count and can be stratified per class
#         )
#         return rf
    
    
    
#     def run_RF_feature_selection(df, fclass, rf=None):
#         """
#         Runs feature selection using a RandomForestClassifier.
    
#         This function iterates over features and samples defined in `fclass`, subsets the DataFrame `df` based on these features and samples, and optionally uses a provided RandomForestClassifier `rf` or creates a default one to fit the data. It outputs the final 
    
#         Parameters:
#         - df (pd.DataFrame): The DataFrame containing the features and target variables.
#         - fclass: An object that must have `feature_dict` and `sample_dict` attributes, and methods `ordered_hierarchy_keys` and `_next_level`. These are used to define the features and samples for subsets and the target variable for the RandomForestClassifier.
#         - rf (RandomForestClassifier, optional): A pre-initialized RandomForestClassifier instance. If None, a new classifier is created with predefined settings.
    
#         Returns:
#         - RandomForestClassifier: The RandomForestClassifier used in the last iteration of feature selection. If `rf` was provided, this will be the same instance, fitted to the last subset of data.
#         """
#         feature_dict = copy.deepcopy(fclass.feature_dict)
#         sample_dict = copy.deepcopy(fclass.sample_dict)
    
#         RF_models_feature_selection_dict = {}
#         RF_feature_importance_dict = {}
        
#         for key, col_idx in feature_dict.items():
#             row_idx = sample_dict[key]
        
#             # get prediction target for RF for each model and sub model 
#             target_key = [fclass.ordered_hierarchy_keys[0] if key == 'Full_model' else fclass._next_level(key[0])]
            
#             combined_columns = target_key + col_idx.tolist()
#             selected_df = df.loc[row_idx, combined_columns]
    
#             if rf is None:
#                 rf = get_preset_RF()
    
#             X = selected_df[col_idx]  # Feature matrix
#             y = selected_df[target_key].iloc[:, 0]  # Target variable (.iloc[:, 0] to unravel to 1D)
#             rf.fit(X, y)
            
#             RF_models_feature_selection_dict[key] = rf
#             RF_feature_importance_dict[key] = rf.feature_importances_
            
#         return RF_feature_selection_dict, RF_feature_importance_dict
    
#     def select_top_n_features():

# RF_feature_selection_dict, RF_feature_importance_dict = run_RF_feature_selection(df, fclass, rf=None)
# # Feature importances from the model
# # feature_importances = rf.feature_importances_






        

In [154]:
_ = [print(k) for k in RF_feature_selection_dict.keys()]

Full_model
('Superfamily_code', 'CNTRL')
('Superfamily_code', 'CRAN_PARASPIN')
('Superfamily_code', 'EMBRY')
('Superfamily_code', 'GLIOMA_GLIONEUR_NEUR')
('Superfamily_code', 'HEMA')
('Superfamily_code', 'MELANO')
('Superfamily_code', 'MESEN_NONMENING')
('Superfamily_code', 'PIN_RET')
('Superfamily_code', 'SELLAR')
('Family_code', 'CIRC_ASTR_GLIO')
('Family_code', 'CNTL_BRAIN')
('Family_code', 'CRANIO')
('Family_code', 'CTRL_TME')
('Family_code', 'EPENDYM')
('Family_code', 'ESTHESIO')
('Family_code', 'GLIONEUR_NEUR')
('Family_code', 'HGG')
('Family_code', 'MB')
('Family_code', 'OTHER_EMBRY')
('Family_code', 'PAP_PINEAL')
('Family_code', 'PINEOBLAST')
('Family_code', 'PIT_AD')
('Family_code', 'PLEXUS')
('Family_code', 'SFT')
('Family_code', 'UNCERT_DIFF')


In [168]:
# RF_feature_importance_dict

In [134]:

# Assuming `selected_df` is your DataFrame, `target_key` is the name of the target variable,
# and `col_idx` is the list or Index object of feature names

X = selected_df[col_idx]  # Feature matrix
y = selected_df[target_key].iloc[:, 0]  # Target variable (.iloc[:, 0] to unravel to 1D)

# Initialize RandomForestClassifier with all the default settings explicitly defined (as of version 0.24.1)
rf = RandomForestClassifier(
    n_estimators=500,                                 # matches the R code provided
    criterion='gini',                                 # matches the R code provided, default for classification 
    max_depth=None,                 # no equivalent in the R code provided
    min_samples_split=2,            # no equivalent in the R code provided
    min_samples_leaf=1,                               # no equivalent in the R code provided -- set to 1 so it is irrelivent 
    min_weight_fraction_leaf=0.0,   # no equivalent in the R code provided
    max_features='sqrt',                              # same as "mtry" in R, "sqrt(p)" is default in R 
        max_leaf_nodes=None,                          # same as "maxnode" default in R is NULL eq to None in python
    min_impurity_decrease=0.0,                        # no equivalent in the R, set to 0 to match R (I assume since there is no param for this)
    bootstrap=True,                                   # consistant with R when set to True: when R code "replace" is set to True
    oob_score=False,                                  # for Python it is default accuracy (but can be customized, for R param "err.rate" is kinda the same but not sure which metric
    n_jobs=None,                     # num jobs run in parallel
    random_state=42,                 # equivalent is set.seed -- not set in R code provided 
    verbose=0,                       # 
    warm_start=False,                # no equivalent in the R, default in python is False
    class_weight=None,               # no equivalent in the R, it would be wise to use this for our data
    ccp_alpha=0.0,                   # no equivalent in the R, default in python is 0 
    max_samples=None                 # no equivalent in the R, realted to "sampsize", instead of fraction it is a count and can be stratified per class
)

"""
Alex RF code notes 
sampsize is not set the default is used

"""

# Fit the model
rf.fit(X, y)

# Feature importances from the model
feature_importances = rf.feature_importances_

# Since Scikit-learn's RandomForestClassifier does not directly provide gain and times used importances,
# `feature_importances_` gives the mean decrease in impurity (Gini importance) by default.
# For more detailed feature importances like gain or times used, consider using a tree interpretation library
# or examining additional output from ensemble methods.


In [137]:
feature_importances

array([0.04435318, 0.19276181, 0.09055441, 0.0523614 , 0.20708419,
       0.07756674, 0.03963039, 0.15400411, 0.07145791, 0.07022587])

In [135]:
nil.info(rf)

type is <class 'sklearn.ensemble._forest.RandomForestClassifier'> will try printing using "get_class_info2" 
apply                    type->   method                                        None  ->   None   ...fier(n_estimators=500, random_state=42)>
bootstrap                type->   bool                                          None  ->   None   > True
ccp_alpha                type->   float                                         None  ->   None   > 0.0
class_weight             type->   NoneType                                      None  ->   None   > None
classes_                 type->   numpy.ndarray                                 shape ->   (2,)   > ['CNS_SARC_CIC' 'EWS']
criterion                type->   str                                           length->   4      > gini
decision_path            type->   method                                        None  ->   None   ...fier(n_estimators=500, random_state=42)>
estimator                type->   sklearn.tree._classes.DecisionT

In [136]:
score.score

NameError: name 'score' is not defined

In [80]:
col_idx, Keys.targets

(Index(['cg00698771', 'cg09814029', 'cg00590173', 'cg03078363', 'cg11165254',
        'cg17090968', 'cg10623600', 'cg03094555', 'cg02928365', 'cg01273336'],
       dtype='object'),
 ['Superfamily_code', 'Family_code', 'Class_v12_code'])

In [109]:
fclass.ordered_hierarchy_keys

(('Family_code', 'UNCERT_DIFF'),
 Index(['cg00698771', 'cg09814029', 'cg00590173', 'cg03078363', 'cg11165254',
        'cg17090968', 'cg10623600', 'cg03094555', 'cg02928365', 'cg01273336'],
       dtype='object'))

In [83]:
print(key)
combined_columns = Keys.targets + col_idx.tolist()
selected_df = df.loc[row_idx, combined_columns]
selected_df
"""
TOMORROW 
add a one to the right as a function. use it to input family code and then output the one to the right. if none use superfamily etc 

"""

('Family_code', 'UNCERT_DIFF')


,Superfamily_code,Family_code,Class_v12_code,cg00698771,cg09814029,cg00590173,cg03078363,cg11165254,cg17090968,cg10623600,cg03094555,cg02928365,cg01273336
93,MESEN_NONMENING,UNCERT_DIFF,CNS_SARC_CIC,0.051607,0.114027,0.135960,0.917757,0.205136,0.974512,0.961461,0.077338,0.053894,0.763298
116,MESEN_NONMENING,UNCERT_DIFF,CNS_SARC_CIC,0.977938,0.153343,0.340490,0.030035,0.126417,0.062431,0.809229,0.138272,0.056809,0.152960
138,MESEN_NONMENING,UNCERT_DIFF,CNS_SARC_CIC,0.000376,0.194952,0.810271,0.060946,0.132074,0.116152,0.099051,0.224423,0.707510,0.340458
210,MESEN_NONMENING,UNCERT_DIFF,EWS,0.953946,0.926938,0.860391,0.021264,0.914232,0.835410,0.893438,0.761164,0.735702,0.889978
211,MESEN_NONMENING,UNCERT_DIFF,EWS,0.884258,0.879832,0.090083,0.583317,0.706367,0.920827,0.849094,0.925970,0.135753,0.228689
238,MESEN_NONMENING,UNCERT_DIFF,EWS,0.975603,0.848003,0.872072,0.730059,0.811057,0.850679,0.062948,0.652688,0.479099,0.848466


0      False
1      False
2      False
3      False
4      False
       ...  
268    False
269    False
270    False
271    False
272    False
Name: Family_code, Length: 273, dtype: bool

In [51]:
df[sample_dict]

,cg04209913,cg24981593,cg10623600,cg01207684,cg25286482,cg06132069,cg19685398,cg17090968,cg08684879,cg10193711
0,0.957660,0.880206,0.892596,0.689138,0.437610,0.881157,0.871105,0.903203,0.877601,0.064660
1,0.479484,0.549356,0.924873,0.901707,0.787077,0.849258,0.876959,0.907800,0.872781,0.136347
2,0.170831,0.780415,0.960258,0.964254,0.747812,0.834966,0.933284,0.821747,0.916791,0.347642
3,0.032742,0.878653,0.248598,0.649712,0.954239,0.882828,0.882949,0.151583,0.911286,0.065328
4,0.051339,0.882442,0.000000,0.971505,0.273014,0.887106,0.840386,0.936575,0.855146,0.556122
...,...,...,...,...,...,...,...,...,...,...
268,0.525080,0.110503,0.184995,0.554307,0.368329,0.353614,0.421679,0.389242,0.707935,0.846112
269,0.569836,0.156192,0.223783,0.527383,0.341912,0.267253,0.413058,0.364666,0.703106,0.816916
270,0.058847,0.554293,0.612695,0.908910,0.257773,0.822999,0.332798,0.158572,0.105684,0.230585
271,0.097293,0.908827,0.092501,0.929232,0.400905,0.821034,0.090114,0.129474,0.207463,0.563816


In [25]:
hierarchy_key = 'Superfamily_code'
hierarchy_name = 'EMBRY'
selected_rows_index = df[hierarchy_key] ==hierarchy_name

selected_rows_index = slice(None)
df.loc[selected_rows_index]

,Superfamily_code,Family_code,Class_v12_code,cg00212031,cg00650640,cg02010442,cg03272642,cg04413754,cg05467600,cg06443675,...,cg20787201,cg21432763,cg22117819,cg22764925,cg23568913,cg24300216,cg25246692,cg25947555,cg26730347,cg27454842
0,EMBRY,MB,MB_SHH_CHL_AD,0.060056,0.174830,0.069681,0.014120,0.323789,0.001262,0.051747,...,0.441887,0.777161,0.615736,0.107996,0.051777,0.867771,0.144181,0.083772,0.346352,0.639290
1,EMBRY,MB,MB_SHH_INF,0.105263,0.000000,0.022624,0.038429,0.364521,0.014286,0.103865,...,0.542503,0.805603,0.405675,0.140824,0.031228,0.905519,0.107303,0.054950,0.229818,0.557485
2,EMBRY,MB,MB_G3,0.066739,0.000000,0.084729,0.017830,0.314324,0.011889,0.036709,...,0.834260,0.770924,0.325451,0.114108,0.035967,0.909144,0.128023,0.053382,0.626938,0.427096
3,EMBRY,MB,MB_G4,0.531080,0.063247,0.005567,0.563444,0.217828,0.049134,0.430767,...,0.754954,0.770129,0.084212,0.079013,0.000000,0.840391,0.070619,0.026671,0.222368,0.451694
4,EMBRY,MB,MB_G3,0.043150,0.024174,0.016813,0.047187,0.339644,0.010687,0.100941,...,0.841666,0.298231,0.355325,0.179955,0.034624,0.887187,0.087058,0.050882,0.075250,0.266236
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268,CNTRL,CNTL_BRAIN,CTRL_HYPOTHAL,0.058519,0.179859,0.036976,0.031447,0.213456,0.000087,0.029990,...,0.858359,0.717412,0.655881,0.401737,0.053558,0.922711,0.324555,0.093040,0.590416,0.531673
269,CNTRL,CNTL_BRAIN,CTRL_HYPOTHAL,0.033036,0.070123,0.038106,0.024388,0.284708,0.000090,0.085306,...,0.851719,0.751161,0.631748,0.426520,0.057775,0.916897,0.308583,0.090510,0.495698,0.492520
270,SELLAR,PIT_AD,PITAD_TSH,0.066259,0.181447,0.052543,0.034760,0.302180,0.000118,0.080423,...,0.753122,0.438892,0.472267,0.185292,0.049762,0.529293,0.456113,0.059071,0.839434,0.860006
271,SELLAR,PIT_AD,PITAD_TSH,0.310545,0.025473,0.064488,0.079658,0.226034,0.093056,0.068528,...,0.824229,0.389241,0.655720,0.191878,0.038382,0.800685,0.353596,0.092287,0.534643,0.655044


In [16]:
# Assuming self.hierarchy_keys are the keys to sort and
# self.hierarchy_levels contain the corresponding levels (lower numbers first),
# this code sorts hierarchy_keys based on hierarchy_levels.

hierarchy_keys = ['b', 'c', 'a']
hierarchy_levels = [2, 1, 3]

sorted_keys = [key for _, key in sorted(zip(hierarchy_levels, hierarchy_keys))]
sorted_keys


['c', 'b', 'a']

In [19]:
for lev, key in sorted(zip(hierarchy_levels, hierarchy_keys)):
    print(lev, key)

1 c
2 b
3 a


In [ ]:
import pandas as pd
import numpy as np
from datetime import date

# Assume 'df' is already loaded and available
# Define the utility directory for saving the output files
util_dir = "path_to_util_directory"

def calculate_IQR(betas):
    return betas.apply(np.quantile, q=[0.25, 0.75]).diff().iloc[1]

def write_top_probes(df, identifier, top_n=10000):
    IQR = calculate_IQR(df.drop(columns=identifier))
    top_probes = IQR.sort_values(ascending=False).head(top_n).index
    filename = f"{util_dir}/{date.today()}_{df[identifier].iloc[0]}_cpg_probes.csv"
    pd.DataFrame(top_probes).to_csv(filename, index=False)

# Main function to process the DataFrame
def process_dataframe(df):
    # Calculating IQR for the entire dataset excluding identifier columns
    ref_IQR = calculate_IQR(df.iloc[:, 3:])
    superfamily_probes = ref_IQR.sort_values(ascending=False).head(10000).index
    pd.DataFrame(superfamily_probes).to_csv(f"{util_dir}/{date.today()}_superfamily_probes.csv", index=False)
    
    # Processing for each superfamily code
    for code in df['Superfamily_code'].unique():
        superfamily_df = df[df['Superfamily_code'] == code].drop(columns=['Superfamily_code', 'Family_code', 'Class_v12_code'])
        write_top_probes(superfamily_df, 'Family_code')
    
    # Processing for each family code
    for code in df['Family_code'].unique():
        family_df = df[df['Family_code'] == code].drop(columns=['Superfamily_code', 'Family_code', 'Class_v12_code'])
        write_top_probes(family_df, 'Class_v12_code')

# Call the function
process_dataframe(df)


In [4]:
import pandas as pd
import numpy as np
from datetime import date

def calculate_IQR(betas):
    """Calculate the IQR for each column."""
    return betas.apply(np.quantile, q=[0.25, 0.75]).diff().iloc[1]

def write_top_probes(df, identifier, top_n=10000):
    """Write top probes based on IQR to a CSV file."""
    IQR = calculate_IQR(df.drop(columns=identifier))
    top_probes = IQR.sort_values(ascending=False).head(top_n).index
    filename = f"{date.today()}_{df[identifier].iloc[0]}_cpg_probes.csv"
    pd.DataFrame(top_probes).to_csv(filename, index=False)

def process_dataframe(df):
    """Process the dataframe to find and write top probes based on IQR."""
    # Exclude the first three columns for the initial IQR calculation
    ref_IQR = calculate_IQR(df.iloc[:, 3:])
    superfamily_probes = ref_IQR.sort_values(ascending=False).head(10000).index
    pd.DataFrame(superfamily_probes).to_csv(f"{date.today()}_superfamily_probes.csv", index=False)
    
    # For each superfamily code
    for code in df['Superfamily_code'].unique():
        superfamily_df = df[df['Superfamily_code'] == code].drop(columns=['Superfamily_code', 'Family_code', 'Class_v12_code'])
        write_top_probes(superfamily_df, 'Family_code')
    
    # For each family code
    for code in df['Family_code'].unique():
        family_df = df[df['Family_code'] == code].drop(columns=['Superfamily_code', 'Family_code', 'Class_v12_code'])
        write_top_probes(family_df, 'Class_v12_code')

# Assuming df is your DataFrame
# process_dataframe(df)


In [5]:
# Call the function
process_dataframe(df)

KeyError: "['Family_code'] not found in axis"